In [1]:
# 初始化環境
from rag_to_vector_db_pinecone.rag_pipeline.chunks_embeding_2_vector_db import (
    jina_embedding,
)

import json
from pinecone import Pinecone
from langchain_openai import ChatOpenAI

# 載入 env 設定
with open('env.json', 'r', encoding='utf-8') as file:
    config = json.load(file)

# 初始化 Pinecone gRPC 客戶端
pc = Pinecone(api_key=config['pinecone_api_key'])
index = pc.Index("rag-vector-db-test")

# 初始化 OpenAI 模型
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # 可以改為 gpt-3.5-turbo 測試
    openai_api_key=config["chat_gpt_key"],
    temperature=0.6
)

c:\Users\TMP214\Desktop\GitHub\project_RAG_and_Graph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 使用者輸入查詢 pinecone
def query_pincone(query, config):
    query_embedding = jina_embedding(query, config)
    results = index.query(
        vector=query_embedding,
        top_k=3,
        include_metadata=True
    )

    matches = results['matches']
    # 用 id 匹配資料庫文字
    with open('data/chunks.json', 'r', encoding='utf-8') as file:
        chunks = json.load(file)

    match_chunks_data = []
    for match in matches:
        for chunk in chunks:
            if chunk['id'] == match['id']:
                match_chunks_data.append({
                    "id" : match['id'],
                    "score" : match['score'],
                    "text" : chunk['text']
                })
                break

    return match_chunks_data


def create_augmented_prompt(query, match_chunks_data):
    # 將檢索到的上下文整合成一個字符串
    contexts = ""
    for i, context in enumerate(match_chunks_data):
        contexts += f"參考資料 {i+1} [相關度: {context['score']:.2f}]:\n{context['text']}\n\n"
    
    # 構建結構化提示
    prompt = f"""
    以下是使用者的查詢問題，請根據提供的參考資料回答。如果參考資料中沒有相關資訊，請明確說明。

    使用者查詢: {query}
    請根據以下參考資料提供準確、簡潔且有條理的回答。引用資料時請註明來源編號。

    {contexts}
    """
    
    return prompt


def query_llm(prompt):
    try:
        response = llm.invoke(prompt)
        return response.content
    except:
        return f"LLM 錯誤"

In [4]:
def main():
    # 1. 決定使用者輸入
    query = '黴菌相關風險與控制'

    # 2. 尋找知識庫
    match_chunks_data = query_pincone(query, config)

    # 3. 將 知識庫資料結合至 prompt
    prompt = create_augmented_prompt(query, match_chunks_data)
    
    # 4. prompt 引入 LLM
    rag_result = query_llm(prompt)
    result = query_llm(query)       # 沒有經過 RAG 檢索
    
    # 5. 對比rag 與沒有 rag 的狀況
    print('================ RAG result ==========')
    print(rag_result)
    
    print('================ 普通 result ==========')
    print(result)

main()

================ RAG result ==========
根據提供的參考資料，黴菌相關風險與控制可以從以下幾個方面來進行：

1. 黴菌毒素：主要包括黃曲霉毒素、赭曲霉毒素、鐮刀菌毒素、單端孢霉烯族毒素和玉米赤霉烯酮等不同類型的毒素，可以透過色譜分析、免疫分析和生物感測器等方法進行檢測。

2. 防控策略：在預防措施方面，可以控制儲存條件、使用防黴包裝和實行農業良好實踐；去除方法可以包括物理方法、化學方法和生物方法。

總結來說，了解黴菌的生物學特性、分類系統、研究方法和應用技術對於開發新型生物資源、解決實際生產問題和促進產業升級具有重要意義。合理控制黴菌相關風險，實現可持續利用黴菌資源也是未來研究與應用的重要方向。
================ 普通 result ==========
黴菌是一種微生物，常見於環境中的空氣、土壤和水中。雖然大部分的黴菌對人體無害，但有些黴菌會釋放出有害物質，對人體健康造成危害。以下是一些與黴菌相關的風險以及相應的控制措施：

1. 健康風險：黴菌在潮濕、溫暖的環境中生長迅速，可能導致室內空氣中的黴菌濃度升高，進而引發呼吸道疾病、過敏反應等健康問題。

控制措施：保持室內環境通風良好，保持室內乾燥，定期清潔和除去潮濕的地方，避免食物和其他有機物質長時間暴露在潮濕環境中。

2. 食品安全風險：某些黴菌會在食物中生長，產生有毒物質，食用後可能導致中毒。

控制措施：保持食品清潔衛生，避免食用發霉的食物，注意保存食物的方式，避免食物受潮。

3. 製造業風險：在某些製造業中，如食品加工、製藥等，可能存在黴菌污染的風險，影響產品質量和安全。

控制措施：建立適當的衛生管理制度，確保生產過程中的衛生條件和控制措施，定期進行衛生檢查和監測。

總的來說，要有效控制黴菌相關風險，需要從源頭上加強管理，保持環境清潔和乾燥，注意食品安全，建立適當的衛生管理制度，以確保人體健康和產品安全。
